In [80]:
import pandas as pd

## Charity Regulator Data

In [111]:
ann_reports = pd.read_csv("../Charity_Data/annual-reports-15012021.csv")
pub_reg = pd.read_csv("../Charity_Data/public-register-15012021.csv").drop(['Unnamed: 11'], axis=1)

all_regulator = ann_reports.merge(pub_reg, on='Registered Charity Number', how='left').drop(['Registered Charity Name_y'], axis=1).rename(columns={"Registered Charity Name_x": "Registered Charity Name"})

In [112]:
all_regulator.loc[(all_regulator['Report Activity'].str.contains("Religious activities") | all_regulator['Beneficiaries'].str.contains("Religious|Religion|Priests")) & all_regulator['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of religion"

all_regulator.loc[all_regulator['Report Activity'].str.contains("Promotion of community") & all_regulator['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of community development, including rural or urban regeneration"

all_regulator.loc[all_regulator['Report Activity'].str.contains("Promotion of health|Relief of poverty") & all_regulator['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Promotion of health, including the prevention or relief of sickness, disease or human suffering"

all_regulator.loc[all_regulator['Report Activity'].str.contains("Advancement of Arts") & all_regulator['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of the arts, culture, heritage or sciences"

all_regulator.loc[all_regulator['Report Activity'].str.contains("Animal welfare") & all_regulator['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Prevention or relief of suffering of animals"

all_regulator.loc[(all_regulator['Report Activity'].str.contains("Welfare/benevolent|Provision of accommodation/housing|Welfare of those in need|Disability support")) & all_regulator['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of community welfare including the relief of those in need by reason of youth, age, ill-health, or disability"


all_regulator.loc[(all_regulator['Report Activity'].str.contains("Education|Research/evaluation|Playgroup/afterschool") | all_regulator['Beneficiaries'].str.contains("University|School|College")) & all_regulator['Charitable Purpose'].isnull(), "Charitable Purpose"] = "Advancement of education"

In [120]:
all_regulator.count()

Registered Charity Number                                         36339
Registered Charity Name                                           36339
Report Size                                                       36339
Period Start Date                                                 36339
Period End Date                                                   36339
Report Activity                                                   36339
Activity Description                                              36335
Beneficiaries                                                     36339
Financial: Income from Central Government or Local Authorities     8164
Financial: Income from other public bodies                         4286
Financial: Income from philantrophic organisations                 1554
Financial: Income from donations                                   9523
Financial: Income from trading and commercial activities           5973
Financial: Income from other sources                            